In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
seir_data = pd.read_csv(
    "./csvs/model_2_1a_seir.csv",
    index_col=0,
)
config = {
    "tot_popn": 1e5,
    "infectious_seed": 1.,
    "end_time": 1000.,
}
compartments = (
    "Susceptible", 
    "Pre-infectious", 
    "Infectious", 
    "Immune"
)
model = CompartmentalModel(
    times=(0., config["end_time"]),
    compartments=compartments,
    infectious_compartments=("Infectious",),
)
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infectious_seed"],
        "Infectious": config["infectious_seed"],
    }
)
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("R0") / ave_infous,
    source="Susceptible",
    dest="Pre-infectious"
)
model.add_transition_flow(
    name="progression", 
    fractional_rate=1. / Parameter("ave_preinfous"),
    source="Pre-infectious", 
    dest="Infectious"
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune"
)
parameters = {
    "ave_preinfous": 2.,
    "ave_infous": 2.,
    "R0": 2.,
}
model.run(parameters=parameters, solver="euler")
seir_outputs = model.get_outputs_df()
assert abs(seir_data - seir_outputs).max().max() < 1e-5

In [ ]:
sir_data = pd.read_csv(
    "./csvs/model_2_1a_sir.csv",
    index_col=0,
)
compartments = ("Susceptible", "Infectious", "Immune")
model = CompartmentalModel(
    times=(0., config["end_time"]),
    compartments=compartments,
    infectious_compartments=["Infectious"],
)
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infectious_seed"],
        "Infectious": config["infectious_seed"],
    }
)
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("R0") / ave_infous,
    source="Susceptible", 
    dest="Infectious"
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune"
)
model.run(parameters=parameters, solver="euler")
sir_outputs = model.get_outputs_df()
assert abs(sir_data - sir_outputs).max().max() < 1e-5